In [1]:
# Install the necessary libraries
%pip install tensorflow
%pip install albumentations
%pip install matplotlib
%pip install scikit-learn
%pip install opencv-python
%pip install opencv-python-headless
%pip install opencv-contrib-python
%pip install tqdm
# Importing the necessary libraries
import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import CSVLogger  # Import Keras from TensorFlow
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score
from ModelArchitecture.DiceLoss import dice_metric_loss
from ModelArchitecture import DUCK_Net
from ImageLoader import ImageLoader2D

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

/Users/majjarisuman/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/majjarisuman/Library/Python/3.9/lib/python/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.16). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:

# Checking the number of GPUs available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Checking the TensorFlow version
print("TensorFlow version:", tf.__version__)

Num GPUs Available:  0
TensorFlow version: 2.17.0


In [3]:
# Setting the model parameters

img_size = 352
dataset_type = 'kvasir' # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 1e-4
seed_value = 58800
filters = 17 # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

ct = datetime.now()

model_type = "DuckNet"

progress_path = 'ProgressFull/' + dataset_type + '_progress_csv_' + model_type + '_filters_' + str(filters) +  '_' + str(ct) + '.csv'
progressfull_path = 'ProgressFull/' + dataset_type + '_progress_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.txt'
plot_path = 'ProgressFull/' + dataset_type + '_progress_plot_' + model_type + '_filters_' + str(filters) + '_' + str(ct) + '.png'
model_path = 'ModelSaveTensorFlow/' + dataset_type + '/' + model_type + '_filters_' + str(filters) + '_' + str(ct)

EPOCHS = 1
min_loss_for_saving = 1

In [4]:
# Loading the data

X, Y = ImageLoader2D.load_data(img_size, img_size, -1, 'Kvasir-SEG')

Total number of images found: 800
Resizing training images and masks: 800


800it [00:03, 243.39it/s]


In [5]:
# Splitting the data, seed for reproducibility
if X.size > 0 and Y.size > 0:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of Y: {Y.shape}")
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=seed_value)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.111, shuffle=True, random_state=seed_value)
else:
    print("Dataset is empty. Cannot perform train-test split.")

Shape of X: (800, 352, 352, 3)
Shape of Y: (800, 352, 352, 1)


In [6]:
# Defining the augmentations

aug_train = albu.Compose([
    albu.HorizontalFlip(),
    albu.VerticalFlip(),
    albu.ColorJitter(brightness=(0.6,1.6), contrast=0.2, saturation=0.1, hue=0.01, always_apply=True),
    albu.Affine(scale=(0.5,1.5), translate_percent=(-0.125,0.125), rotate=(-180,180), shear=(-22.5,22), always_apply=True),
])

def augment_images():
    x_train_out = []
    y_train_out = []

    for i in range (len(x_train)):
        ug = aug_train(image=x_train[i], mask=y_train[i])
        x_train_out.append(ug['image'])  
        y_train_out.append(ug['mask'])

    return np.array(x_train_out), np.array(y_train_out)

In [7]:
# Creating the model

model = DUCK_Net.create_model(img_height=img_size, img_width=img_size, input_chanels=3, out_classes=1, starting_filters=filters)

Starting DUCK-Net


In [8]:
# Compiling the model

model.compile(optimizer=optimizer, loss=dice_metric_loss)

In [9]:
# Training the model

step = 0

for epoch in range(0, EPOCHS):
    
    print(f'Training, epoch {epoch}')
    print('Learning Rate: ' + str(learning_rate))

    step += 1
        
    image_augmented, mask_augmented = augment_images()
    
    csv_logger = CSVLogger(progress_path, append=True, separator=';')
    
    model.fit(x=image_augmented, y=mask_augmented, epochs=1, batch_size=4, validation_data=(x_valid, y_valid), verbose=1, callbacks=[csv_logger])
    
    prediction_valid = model.predict(x_valid, verbose=0)
    loss_valid = dice_metric_loss(y_valid, prediction_valid)
    
    loss_valid = loss_valid.numpy()
    print("Loss Validation: " + str(loss_valid))
        
    prediction_test = model.predict(x_test, verbose=0)
    loss_test = dice_metric_loss(y_test, prediction_test)
    loss_test = loss_test.numpy()
    print("Loss Test: " + str(loss_test))
        
    with open(progressfull_path, 'a') as f:
        f.write('epoch: ' + str(epoch) + '\nval_loss: ' + str(loss_valid) + '\ntest_loss: ' + str(loss_test) + '\n\n\n')
    
    if min_loss_for_saving > loss_valid:
        min_loss_for_saving = loss_valid
        print("Saved model with val_loss: ", loss_valid)
        model.save(model_path + '.h5')
        
    del image_augmented
    del mask_augmented

    gc.collect()

Training, epoch 0
Learning Rate: 0.0001
160/160 ━━━━━━━━━━━━━━━━━━━━ 3377s 21s/step - loss: 0.6723 - val_loss: 0.6316
Loss Validation: 0.6212857


Loss Test: 0.65739703
Saved model with val_loss:  0.6212857


In [10]:
# Assuming you have a trained model named 'model'
# Generate predictions for training, testing, and validation datasets
prediction_train = model.predict(x_train)
prediction_test = model.predict(x_test)
prediction_valid = model.predict(x_valid)

# Calculate recall scores
recall_test = recall_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
recall_valid = recall_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                            np.ndarray.flatten(prediction_valid > 0.5))

print("Recall finished")

# Calculate accuracy scores
accuracy_train = accuracy_score(np.ndarray.flatten(np.array(y_train, dtype=bool)),
                                np.ndarray.flatten(prediction_train > 0.5))
accuracy_test = accuracy_score(np.ndarray.flatten(np.array(y_test, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
accuracy_valid = accuracy_score(np.ndarray.flatten(np.array(y_valid, dtype=bool)),
                                np.ndarray.flatten(prediction_valid > 0.5))

print("Accuracy finished")

# Construct the final file name
final_file = 'results_' + model_type + '_' + str(filters) + '_' + dataset_type + '.txt'
print(final_file)

# Write results to the file
with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write('recall_test: ' + str(recall_test) + ' recall_valid: ' + str(recall_valid) + '\n\n')
    f.write('accuracy_train: ' + str(accuracy_train) + ' accuracy_test: ' + str(accuracy_test) + ' accuracy_valid: ' + str(accuracy_valid) + '\n\n')

20/20 ━━━━━━━━━━━━━━━━━━━━ 556s 28s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 69s 21s/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 77s 22s/step
Recall finished
Accuracy finished
results_DuckNet_17_kvasir.txt


In [20]:
# Open and read the file
file_path = 'results_DuckNet_17_kvasir.txt'

# Reading the file and printing its content
with open(file_path, 'r') as file:
    content = file.read()

# Print the contents of the file
print(content)

kvasir

recall_test: 0.5258511999331694 recall_valid: 0.5346469493543626

accuracy_train: 0.7659094440050361 accuracy_test: 0.7582656734245867 accuracy_valid: 0.776212228822314


